In [1]:
import os
import bigjson

In [6]:
question_path = os.path.join(
    os.getenv("STORAGE"),
    "datasets",
    "GQA",
    "questions"
)

In [7]:
os.listdir(question_path)

['challenge_all_questions.json',
 'challenge_balanced_questions.json',
 'minitest.json',
 'readme.txt',
 'submission_all_questions.json',
 'testdev_all_questions.json',
 'testdev_balanced_questions.json',
 'test_all_questions.json',
 'test_balanced_questions.json',
 'train_all_questions',
 'train_balanced_questions.json',
 'val_all_questions.json',
 'val_balanced_questions.json']

In [13]:
q_file = os.path.join(question_path, "train_balanced_questions.json")

with open(q_file, "rb") as f:
    data = bigjson.load(f)
    print(data["02930152"])

In [14]:
images_path = os.path.join(
    os.getenv("STORAGE"),
    "datasets",
    "GQA",
    "images"
)

image_names = os.listdir(images_path)
len(image_names)

148854

In [15]:
image_names[0]

'1.jpg'

In [16]:
# pick one image
image = image_names[23]
image_id = image.split(".")[0]
image_id

'102'

So the problem is that the question object holds the image id. Not the other way around. (Since image is grounded by text!)

In [19]:
import pandas as pd

%time df = pd.read_json(q_file)

CPU times: user 2min 55s, sys: 2.77 s, total: 2min 58s
Wall time: 2min 58s


In [21]:
df = df.T
df.head()

,semantic,entailed,equivalent,question,imageId,isBalanced,groups,answer,semanticStr,annotations,types,fullAnswer
2930152,"[{'operation': 'select', 'dependencies': [], '...","[02930160, 02930158, 02930159, 02930154, 02930...",[02930152],Is the sky dark?,2354786,True,"{'global': None, 'local': '06-sky_dark'}",yes,select: sky (2486325)->verify color: dark [0],"{'answer': {}, 'question': {'2': '2486325'}, '...","{'detailed': 'verifyAttr', 'semantic': 'attr',...","Yes, the sky is dark."
7333408,"[{'operation': 'select', 'dependencies': [], '...",[],[07333408],What is on the white wall?,2375429,True,"{'global': '', 'local': '14-wall_on,s'}",pipe,select: wall (722332)->filter color: white [0]...,"{'answer': {'0': '722335'}, 'question': {'4:6'...","{'detailed': 'relS', 'semantic': 'rel', 'struc...",The pipe is on the wall.
7333405,"[{'operation': 'select', 'dependencies': [], '...",[07333406],[07333405],Is that pipe red?,2375429,True,"{'global': None, 'local': '06-pipe_red'}",no,select: pipe (722335)->verify color: red [0],"{'answer': {}, 'question': {'2': '722335'}, 'f...","{'detailed': 'verifyAttrC', 'semantic': 'attr'...","No, the pipe is white."
15736264,"[{'operation': 'select', 'dependencies': [], '...","[15736259, 15736258, 15736267, 15736253, 15736...",[15736264],Is the tall clock small or large?,2368326,True,"{'global': 'size', 'local': '10c-clock_size'}",large,select: clock (746851)->filter height: tall [0...,"{'answer': {}, 'question': {'2:4': '746851'}, ...","{'detailed': 'chooseAttr', 'semantic': 'attr',...",The clock is large.
111007521,"[{'operation': 'select', 'dependencies': [], '...",[],[111007521],Who is wearing a shirt?,2331819,True,"{'global': 'person', 'local': '14-shirt_wearin...",girl,"select: shirt (4653737)->relate: person,wearin...","{'answer': {'0': '4653736'}, 'question': {'4':...","{'detailed': 'relS', 'semantic': 'rel', 'struc...",The girl is wearing a shirt.


Loading all that altogether means a lot of memory consumption. How about lazy loading with dask? Should work! 